In [1]:
import sys
import os
import matplotlib.pyplot as plt
% matplotlib inline
import pickle
# Import class
parent_dir = os.path.dirname(os.path.dirname(os.getcwd()))
class_dir = os.path.join(parent_dir, 'Scripts')
sys.path.insert(0, class_dir)
import NBAStats
import numpy as np
import datetime as dt
import random
import seaborn as sns
import pandas as pd
import statsmodels.api as sm

stats_file = os.path.join(parent_dir, 'Data', 'nba_stats.pickle')
print(stats_file)
nba_stats = pickle.load(open(stats_file, 'rb'))

C:\Users\johnn\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


C:\Users\johnn\Desktop\github\nba_stats\Data\nba_stats.pickle


# Christmas game effect
Do players play differently on Christmas day?
For each player who's played on christmas day, compare how they've performed against their previous 100 game average.

In [2]:
year = 2017

christmas_day = dt.date(year, 12, 25)
games_info = nba_stats.get_games().copy()

# Convert GAME_DATE from str to dt.date
games_info['GAME_DATE'] = games_info['GAME_DATE'].apply(
    lambda date: dt.datetime.strptime(date[:10], '%Y-%m-%d'))

xmas_games = games_info[games_info['GAME_DATE'] == christmas_day]
xmas_game_ids = xmas_games['GAME_ID'].unique()

In [3]:
game_id = xmas_game_ids[0]

game_boxscore = nba_stats.get_boxscores()[game_id]
game_boxscore

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0021700497,1610612750,MIN,Minnesota,203952,Andrew Wiggins,F,,34:11,7.0,...,0.0,4.0,4.0,5.0,0.0,0.0,1.0,1.0,16.0,11.0
1,0021700497,1610612750,MIN,Minnesota,201959,Taj Gibson,F,,34:16,11.0,...,4.0,5.0,9.0,0.0,1.0,0.0,1.0,2.0,23.0,5.0
2,0021700497,1610612750,MIN,Minnesota,1626157,Karl-Anthony Towns,C,,39:49,7.0,...,0.0,10.0,10.0,2.0,1.0,4.0,3.0,2.0,21.0,18.0
3,0021700497,1610612750,MIN,Minnesota,202710,Jimmy Butler,G,,41:03,11.0,...,1.0,4.0,5.0,8.0,2.0,0.0,2.0,0.0,23.0,17.0
4,0021700497,1610612750,MIN,Minnesota,201952,Jeff Teague,G,,32:52,3.0,...,0.0,7.0,7.0,10.0,2.0,0.0,5.0,4.0,6.0,9.0
5,0021700497,1610612750,MIN,Minnesota,203476,Gorgui Dieng,,,16:10,1.0,...,1.0,1.0,2.0,0.0,0.0,0.0,1.0,2.0,5.0,13.0
6,0021700497,1610612750,MIN,Minnesota,1626145,Tyus Jones,,,15:08,2.0,...,0.0,3.0,3.0,5.0,2.0,0.0,1.0,0.0,7.0,8.0
7,0021700497,1610612750,MIN,Minnesota,2037,Jamal Crawford,,,19:13,7.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,19.0,11.0
8,0021700497,1610612750,MIN,Minnesota,202357,Nemanja Bjelica,,,5:45,0.0,...,1.0,1.0,2.0,0.0,0.0,0.0,0.0,2.0,1.0,-2.0
9,0021700497,1610612750,MIN,Minnesota,1627875,Marcus Georges-Hunt,,,1:33,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0
